<a href="https://colab.research.google.com/github/Plutosss/Network-Distributed-Algorithm/blob/master/paper2_diff_alg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf /content/sample_data

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Network-Distributed-Algorithm-master')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
#读取文件
dataFile = 'pd_speech_features.csv'
data =  pd.read_csv(dataFile)
data = shuffle(data)
features = data.loc[:, 'gender':'tqwt_kurtosisValue_dec_36']
print('特征里是否有Nan值:',features.isnull().values.any())

# 数据处理
X_total = (features - features.mean()) / (features.std())
X_total['const'] = np.ones(features.shape[0]) #常数列
X_total = X_total.to_numpy()
Y_total = data.iloc[:, -1].values
#print(X_total[:5],Y_total[:5])


m, dim = X_total.shape
print(m,dim)

n_agent = 12
kappa =10
mu = 5e-5  # we set mu=5e-10 when kappa = 10

$tmp = X\ total@w$

$f(w) = \sum \big((Y\ total - 1) * tmp - np.log(1 + np.exp(-tmp))
                    \big) /m\ total + sum(w**2) * LAMBDA / 2$

$g(w) = X\ total.T @ (logit(X\ total, w) - Y\ total) / m\ total + w * LAMBDA $

In [ ]:
from problems import *
p = SparseLogisticRegression(n_agent, m, dim, X_total=X_total, Y_total=Y_total, prob=0.3) #修改了f_min 和x_min节省时间
#print('alpha = ' + str(alpha))
#print(W.sum(axis=0))
#print(W.sum(axis=1))
print('Lips:',p.L)
print('mu:',p.sigma)

In [ ]:
p.x_min.shape

In [ ]:
import random
from optimizers.utils import generate_mixing_matrix, generate_R_matrix, generate_C_matrix
W, alpha = generate_mixing_matrix(p)
R, alpha = generate_R_matrix(p)
C, alpha = generate_C_matrix(p)


# 步长使用1e-4 误差能精确到 
eta_1 = 1 / p.L
eta_2 = 2 / (p.L + p.sigma)

eig_min = np.min(np.abs(np.linalg.eigvals(W)))
print('最小特征值:',eig_min)


In [ ]:
!rm -rf /content/drive/MyDrive/Network-Distributed-Algorithm-master/data/*
!rm -rf /content/drive/MyDrive/Network-Distributed-Algorithm-master/figs/*

In [ ]:
random.seed(1)
x_0 = np.random.rand(dim, n_agent)+0.5*np.repeat(np.expand_dims(p.x_min,axis=1),n_agent,axis=1)

from optimizers import *
from utils import run_exp


n_iters = 1000
n_dgd_iters = n_iters * 1

distributed = [
    DSGD(p, n_iters=n_dgd_iters, eta=eta_1, x_0=x_0, W=W, diminish=True, verbose=True),
    #EXTRA(p, n_iters=n_dgd_iters, eta=0.01, x_0=x_0, W=W), # or p.L/624 the step size bound by eig_min/p.L, proved pushi 
    

    ]
RC = [
    PushDIGing(p, n_iters=n_dgd_iters, eta=0.025, x_0=x_0, R=R, C=C),
    DGD_tracking(p, n_iters=n_dgd_iters, eta=0.03, x_0=x_0, W=W),
    PushPull(p, n_iters=n_dgd_iters, eta=0.03, x_0=x_0, R=R, C=C),
    PEXTRA(p, n_iters=n_dgd_iters, eta=0.04, x_0=x_0, W=W),
    PGPushPull(p, n_iters=n_dgd_iters, eta=0.04, x_0=x_0, R=R, C=C)#突出PGPushPull
    ]
exps = distributed + RC
res = run_exp(exps, kappa=kappa, max_iter=n_iters, name='sparse_logistic_regression', n_process=1, save=True)



In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/Network-Distributed-Algorithm-master/figs/sparse_logistic_regression_kappa_10_fval_iter.eps')
files.download('/content/drive/MyDrive/Network-Distributed-Algorithm-master/figs/sparse_logistic_regression_kappa_10_var_iter.eps')
